# Lab | Langchain Evaluation

## Intro

Pick different sets of data and re-run this notebook. The point is for you to understand all steps involve and the many different ways one can and should evaluate LLM applications.

What did you learn? - Let's discuss that in class

## LangChain: Evaluation

### Outline:

* Example generation
* Manual evaluation (and debuging)
* LLM-assisted evaluation

In [4]:
!pip install dotenv

In [ ]:
from dotenv import load_dotenv, find_dotenv
import os
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv("my key")

### Example 1

#### Create our QandA application

In [9]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [7]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.52
    Uninstalling langchain-core-0.3.52:
      Successfully uninstalled langchain-core-0.3.52


In [11]:
!pip install langchain_huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [1]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.llms import OpenAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import CSVLoader, TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.chains import LLMChain


In [2]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

In [2]:
!pip install numpy==1.26.4 packaging==24.0


In [3]:
!pip install --upgrade --force-reinstall sentence-transformers

  Using cached sentence_transformers-4.1.0-py3-none-any.whl.metadata (13 kB)
  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached scikit_learn-1.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached huggingface_hub-0.30.2-py3-none-any.whl.metadata (13 kB)
  Using cached pillow-11.2.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (8.9 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux20

In [1]:
# ✅ Step 1: Install required libraries
!pip install langchain langchain-huggingface chromadb

# ✅ Step 2: Import libraries
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader

# ✅ Step 3: Load the file you already uploaded (change if needed)
loader = TextLoader("nyc_text (1).txt")
docs = loader.load()

# ✅ Step 4: Setup embedding
embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"}
)

# ✅ Step 5: Create a Chroma vectorstore
db = Chroma.from_documents(docs, embedding=embedding)

# ✅ Step 6: Example query
query = "What is New York famous for?"
results = db.similarity_search(query, k=3)

# ✅ Step 7: Display results
for i, res in enumerate(results, 1):
    print(f"\n🔹 Result {i}:\n{res.page_content}")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



🔹 Result 1:
New York, often called New York City or NYC, is the most populous city in the United States. With a 2020 population of 8,804,190 distributed over 300.46 square miles (778.2 km2), New York City is the most densely populated major city in the United States and more than twice as populous as Los Angeles, the nation's second-largest city. New York City is located at the southern tip of New York State. It constitutes the geographical and demographic center of both the Northeast megalopolis and the New York metropolitan area, the largest metropolitan area in the U.S. by both population and urban area. With over 20.1 million people in its metropolitan statistical area and 23.5 million in its combined statistical area as of 2020, New York is one of the world's most populous megacities, and over 58 million people live within 250 mi (400 km) of the city. New York City is a global cultural, financial, entertainment, and media center with a significant influence on commerce, health ca

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

import os
from getpass import getpass

# # Step 1: API Key
# os.environ["OPENAI_API_KEY"] = getpass("🔐 Enter your OpenAI API key: ")

# Step 2: Load the text file
loader = TextLoader("nyc_text (1).txt")
docs = loader.load()

# Step 3: Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(docs)

# Step 4: Create embeddings and vectorstore
embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"}
)
vectorstore = Chroma.from_documents(split_docs, embedding=embedding)

# Step 5: Set up LLM and Retrieval QA
llm = ChatOpenAI(temperature=0.0)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    verbose=True,
    chain_type_kwargs={"document_separator": "<<<<>>>>>"},
)

# Step 6: Ask your question
question = "What is New York City known for?"
answer = qa.invoke({"query": question})

# Step 7: Output the result
print("🧠 Answer:\n", answer)


🔐 Enter your OpenAI API key: ··········


> Entering new RetrievalQA chain...

> Finished chain.
🧠 Answer:
 {'query': 'What is New York City known for?', 'result': 'New York City is known for being a global cultural, financial, entertainment, and media center. It is also famous for its iconic landmarks such as the Statue of Liberty, Times Square, Central Park, Broadway theaters, and the Empire State Building. Additionally, New York City is renowned for its diverse population, world-class museums, vibrant arts scene, and being a hub for fashion and cuisine.'}


#### Coming up with test datapoints

In [13]:
data[2]

'cherry'

In [15]:
data[1]

'banana'

#### Hard-coded examples

In [19]:
from langchain.prompts import PromptTemplate

In [21]:
from langchain.prompts import PromptTemplate
from langchain.schema import BaseOutputParser
from langchain.chains import LLMChain  # ✅ This line is the fix
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI  # Make sure this is included

# Sample examples
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty 850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

# Define the prompt template
prompt_template = PromptTemplate(
    input_variables=["query"],
    template="Examples:\n"
             "1. Query: Do the Cozy Comfort Pullover Set have side pockets?\n"
             "   Answer: Yes\n"
             "2. Query: What collection is the Ultra-Lofty 850 Stretch Down Hooded Jacket from?\n"
             "   Answer: The DownTek collection\n"
             "Query: {query}\n"
             "Answer:"
)

# Define the output model
class Answer(BaseModel):
    answer: str = Field(description="The answer to the query")

# Create the output parser
class AnswerOutputParser(BaseOutputParser):
    def parse(self, text: str) -> Answer:
        answer = text.strip().split("Answer:")[-1].strip()
        return Answer(answer=answer)

# Initialize the LLM
llm = ChatOpenAI()

# Create the LLMChain
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    output_parser=AnswerOutputParser()
)

# Example query
query = "Is the Cozy Comfort Pullover Set available in different colors?"

# Run the chain
result = llm_chain.run({"query": query})

# Print the result
print(result)


<ipython-input-21-424117be9760>:45: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(


answer='Yes, the Cozy Comfort Pullover Set is available in multiple colors such as grey, blue, and pink.'


#### LLM-Generated examples

In [22]:
from langchain.evaluation.qa import QAGenerateChain

In [23]:
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI())

In [24]:
llm_chain = LLMChain(llm=llm, prompt=prompt_template)

In [25]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

/usr/local/lib/python3.11/dist-packages/langchain/chains/llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [26]:
new_examples[0]

{'qa_pairs': {'query': 'What is the significance of the number "0" in the document provided?',
  'answer': 'The number "0" in the document indicates an absence of value or a starting point, serving as a neutral placeholder in mathematical calculations.'}}

In [32]:
data[0]

0

In [28]:
d_flattened = [data['qa_pairs'] for data in new_examples]
d_flattened

[{'query': 'What is the significance of the number "0" in the document provided?',
  'answer': 'The number "0" in the document indicates an absence of value or a starting point, serving as a neutral placeholder in mathematical calculations.'}]

#### Combine examples

In [29]:
# examples += new_example
examples += d_flattened

In [30]:
examples[0]

{'query': 'Do the Cozy Comfort Pullover Set have side pockets?',
 'answer': 'Yes'}

In [33]:
qa.invoke(examples[0]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Do the Cozy Comfort Pullover Set have side pockets?',
 'result': "I don't have information about the Cozy Comfort Pullover Set."}

### Manual Evaluation - Fun part

In [34]:
import langchain
langchain.debug = True

In [35]:
qa.invoke(examples[0]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set have side pockets?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set have side pockets?",
  "context": "Multibillion-dollar heavy rail transit projects under construction in New York City include the Second Avenue Subway, and the East Side Access project.<<<<>>>>>of the Hudson River and the East River, and its southern tip, at the confluence of the two rivers, represents the birthplace of New York City itself. Several small islands also compose part of the borough of Manhattan, including Randalls and Wards Islands, and Roosevelt Island in the East River, and Governors Island and Liberty Island to the south in New York Harbor.<<<<>>>>>Manhattan Island is loosely div

{'query': 'Do the Cozy Comfort Pullover Set have side pockets?',
 'result': "I don't have information about the Cozy Comfort Pullover Set or its specific features like side pockets."}

In [36]:
# Turn off the debug mode
langchain.debug = False

### LLM assisted evaluation

In [37]:
examples += d_flattened

In [38]:
examples

[{'query': 'Do the Cozy Comfort Pullover Set have side pockets?',
  'answer': 'Yes'},
 {'query': 'What collection is the Ultra-Lofty 850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection'},
 {'query': 'What is the significance of the number "0" in the document provided?',
  'answer': 'The number "0" in the document indicates an absence of value or a starting point, serving as a neutral placeholder in mathematical calculations.'},
 {'query': 'What is the significance of the number "0" in the document provided?',
  'answer': 'The number "0" in the document indicates an absence of value or a starting point, serving as a neutral placeholder in mathematical calculations.'}]

In [39]:
predictions = qa.batch(examples)



> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...

> Finished chain.
> Finished chain.


> Finished chain.

> Finished chain.


In [40]:
predictions

[{'query': 'Do the Cozy Comfort Pullover Set have side pockets?',
  'answer': 'Yes',
  'result': "I don't have information about the Cozy Comfort Pullover Set or its specific features."},
 {'query': 'What collection is the Ultra-Lofty 850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection',
  'result': "I don't have the specific information about the collection that the Ultra-Lofty 850 Stretch Down Hooded Jacket is from."},
 {'query': 'What is the significance of the number "0" in the document provided?',
  'answer': 'The number "0" in the document indicates an absence of value or a starting point, serving as a neutral placeholder in mathematical calculations.',
  'result': "I don't know, could you please provide more context or specify which document you are referring to?"},
 {'query': 'What is the significance of the number "0" in the document provided?',
  'answer': 'The number "0" in the document indicates an absence of value or a starting point, serving as a ne

In [41]:
from langchain.evaluation.qa import QAEvalChain

In [42]:
llm = ChatOpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)

In [43]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [44]:
graded_outputs

[{'results': 'INCORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'INCORRECT'}]

In [45]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    # print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

Example 0:
Question: Do the Cozy Comfort Pullover Set have side pockets?
Real Answer: Yes
Predicted Answer: I don't have information about the Cozy Comfort Pullover Set or its specific features.

Example 1:
Question: What collection is the Ultra-Lofty 850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted Answer: I don't have the specific information about the collection that the Ultra-Lofty 850 Stretch Down Hooded Jacket is from.

Example 2:
Question: What is the significance of the number "0" in the document provided?
Real Answer: The number "0" in the document indicates an absence of value or a starting point, serving as a neutral placeholder in mathematical calculations.
Predicted Answer: I don't know, could you please provide more context or specify which document you are referring to?

Example 3:
Question: What is the significance of the number "0" in the document provided?
Real Answer: The number "0" in the document indicates an absence of value or a 

### Example 2
One can also easily evaluate your QA chains with the metrics offered in ragas

In [49]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

# Load your file
loader = TextLoader("nyc_text.txt")

# Use CPU instead of MPS
index = VectorstoreIndexCreator(
    embedding=HuggingFaceEmbeddings(
        model_name="all-MiniLM-L6-v2",
        model_kwargs={'device': 'cpu'}
    )
).from_loaders([loader])

# Initialize LLM
llm = ChatOpenAI(temperature=0)

# QA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=index.vectorstore.as_retriever(),
    return_source_documents=True,
)


/usr/local/lib/python3.11/dist-packages/langchain/indexes/vectorstore.py:171: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


In [50]:
# testing it out

question = "How did New York City get its name?"
result = qa_chain.invoke({"query": question})
result["result"]

'New York City was originally named New Amsterdam by Dutch colonists in 1626. When the city came under British control in 1664, it was renamed New York after King Charles II of England granted the lands to his brother, the Duke of York. The city has been continuously named New York since November 1674.'

In [51]:
result

{'query': 'How did New York City get its name?',
 'result': 'New York City was originally named New Amsterdam by Dutch colonists in 1626. When the city came under British control in 1664, it was renamed New York after King Charles II of England granted the lands to his brother, the Duke of York. The city has been continuously named New York since November 1674.',
 'source_documents': [Document(id='8d2d2df0-5a6d-43bc-a21f-ae90e947f1e8', metadata={'source': 'nyc_text.txt'}, page_content='The city and its metropolitan area constitute the premier gateway for legal immigration to the United States. As many as 800 languages are spoken in New York, making it the most linguistically diverse city in the world. New York City is home to more than 3.2 million residents born outside the U.S., the largest foreign-born population of any city in the world as of 2016.New York City traces its origins to a trading post founded on the southern tip of Manhattan Island by Dutch colonists in approximately 16

Now in order to evaluate the qa system we generated a few relevant questions. We've generated a few question for you but feel free to add any you want.

In [52]:
eval_questions = [
    "What is the population of New York City as of 2020?",
    "Which borough of New York City has the highest population?",
    "What is the economic significance of New York City?",
    "How did New York City get its name?",
    "What is the significance of the Statue of Liberty in New York City?",
]

eval_answers = [
    "8,804,190",
    "Brooklyn",
    "New York City's economic significance is vast, as it serves as the global financial capital, housing Wall Street and major financial institutions. Its diverse economy spans technology, media, healthcare, education, and more, making it resilient to economic fluctuations. NYC is a hub for international business, attracting global companies, and boasts a large, skilled labor force. Its real estate market, tourism, cultural industries, and educational institutions further fuel its economic prowess. The city's transportation network and global influence amplify its impact on the world stage, solidifying its status as a vital economic player and cultural epicenter.",
    "New York City got its name when it came under British control in 1664. King Charles II of England granted the lands to his brother, the Duke of York, who named the city New York in his own honor.",
    "The Statue of Liberty in New York City holds great significance as a symbol of the United States and its ideals of liberty and peace. It greeted millions of immigrants who arrived in the U.S. by ship in the late 19th and early 20th centuries, representing hope and freedom for those seeking a better life. It has since become an iconic landmark and a global symbol of cultural diversity and freedom.",
]

examples = [
    {"query": q, "ground_truths": [eval_answers[i]]}
    for i, q in enumerate(eval_questions)
]

In [53]:
examples

[{'query': 'What is the population of New York City as of 2020?',
  'ground_truths': ['8,804,190']},
 {'query': 'Which borough of New York City has the highest population?',
  'ground_truths': ['Brooklyn']},
 {'query': 'What is the economic significance of New York City?',
  'ground_truths': ["New York City's economic significance is vast, as it serves as the global financial capital, housing Wall Street and major financial institutions. Its diverse economy spans technology, media, healthcare, education, and more, making it resilient to economic fluctuations. NYC is a hub for international business, attracting global companies, and boasts a large, skilled labor force. Its real estate market, tourism, cultural industries, and educational institutions further fuel its economic prowess. The city's transportation network and global influence amplify its impact on the world stage, solidifying its status as a vital economic player and cultural epicenter."]},
 {'query': 'How did New York City

#### Introducing RagasEvaluatorChain

`RagasEvaluatorChain` creates a wrapper around the metrics ragas provides (documented [here](https://github.com/explodinggradients/ragas/blob/main/docs/metrics.md)), making it easier to run these evaluation with langchain and langsmith.

The evaluator chain has the following APIs

- `__call__()`: call the `RagasEvaluatorChain` directly on the result of a QA chain.
- `evaluate()`: evaluate on a list of examples (with the input queries) and predictions (outputs from the QA chain).
- `evaluate_run()`: method implemented that is called by langsmith evaluators to evaluate langsmith datasets.

lets see each of them in action to learn more.

In [54]:
result = qa_chain.invoke({"query": eval_questions[1]})
result["result"]

'Manhattan (New York County) has the highest population density of any borough in New York City.'

In [55]:
key_mapping = {
    "query": "question",
    "result": "answer",
    "source_documents": "contexts"
}

result_updated = {}
for old_key, new_key in key_mapping.items():
    if old_key in result:
        result_updated[new_key] = result[old_key]


In [56]:
result_updated

{'question': 'Which borough of New York City has the highest population?',
 'answer': 'Manhattan (New York County) has the highest population density of any borough in New York City.',
 'contexts': [Document(id='b1219917-bc84-4ee7-8234-c833300b9819', metadata={'source': 'nyc_text.txt'}, page_content="New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houston, and Phoenix, Arizona combined. New York City's populatio

In [57]:
# !pip install --no-cache-dir recordclass

In [58]:
# !pip install ragas==0.1.9

In [61]:
# Step 1: Ensure the correct packages are installed
!pip install ragas fsspec==2025.3.2 --quiet

# Step 2: Import the working evaluation chains and metrics
from ragas.integrations.langchain import EvaluatorChain
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
)

# Step 3: Create evaluator chains
faithfulness_chain   = EvaluatorChain(metric=faithfulness)
answer_rel_chain     = EvaluatorChain(metric=answer_relevancy)
context_recall_chain = EvaluatorChain(metric=context_recall)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 10.5 MB/s eta 0:00:00


1. `__call__()`

Directly run the evaluation chain with the results from the QA chain. Do note that metrics like context_relevancy and faithfulness require the `source_documents` to be present.

In [62]:
# Recheck the result that we are going to validate.
result

{'query': 'Which borough of New York City has the highest population?',
 'result': 'Manhattan (New York County) has the highest population density of any borough in New York City.',
 'source_documents': [Document(id='b1219917-bc84-4ee7-8234-c833300b9819', metadata={'source': 'nyc_text.txt'}, page_content="New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houston, and Phoenix, Arizona combined. New York City's popu

**Faithfulness**

In [65]:
# Use invoke() to run the evaluation properly
eval_result = faithfulness_chain.invoke(result_updated)

# Print the whole result to understand its structure
print(eval_result)

# Safely access the faithfulness score
try:
    print("Faithfulness Score:", eval_result.score)
except AttributeError:
    print("Faithfulness Score:", eval_result.get("faithfulness_score", "Key not found"))


{'question': 'Which borough of New York City has the highest population?', 'answer': 'Manhattan (New York County) has the highest population density of any borough in New York City.', 'contexts': [Document(id='b1219917-bc84-4ee7-8234-c833300b9819', metadata={'source': 'nyc_text.txt'}, page_content="New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houston, and Phoenix, Arizona combined. New York City's population 

High faithfulness_score means that there are exact consistency between the source documents and the answer.

You can check lower faithfulness scores by changing the result (answer from LLM) or source_documents to something else.

In [72]:
# Step 1: Fake result to test faithfulness
fake_result = result.copy()
fake_result["result"] = "we are the champions"

# Step 2: Ensure required keys exist
fake_result["question"] = result.get("question", "What is the query?")
fake_result["answer"] = fake_result["result"]
fake_result["contexts"] = result.get("contexts", ["This is a placeholder context."])

# Step 3: Run faithfulness evaluation
eval_result = faithfulness_chain.invoke(fake_result)

# Step 4: Print the faithfulness score
print("Faithfulness Score:", eval_result.get("faithfulness", "Score not found"))


Faithfulness Score: 0.0


**Context Relevancy**

In [75]:
# Prepare input for evaluation
result_fixed = result.copy()
result_fixed["question"] = result.get("question", "What is the question?")
result_fixed["ground_truth"] = result.get("result", "Expected correct answer")
result_fixed["contexts"] = result.get("contexts", [])

# Run evaluation
eval_result = context_recall_chain.invoke(result_fixed)

# Print the available keys and their values
print("Returned keys and values:")
for k, v in eval_result.items():
    print(f"🔹 {k}: {v}")

# Safely access the score
print("\nContext Recall Score:", eval_result.get("context_recall_score", "⚠️ Score not found"))


Returned keys and values:
🔹 query: Which borough of New York City has the highest population?
🔹 result: Manhattan (New York County) has the highest population density of any borough in New York City.
🔹 source_documents: [Document(id='b1219917-bc84-4ee7-8234-c833300b9819', metadata={'source': 'nyc_text.txt'}, page_content="New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houston, and Phoenix, Arizona combined. New

High context_recall_score means that the ground truth is present in the source documents.

You can check lower context recall scores by changing the source_documents to something else.

In [79]:
from langchain.schema import Document

# Prepare the fake result properly
fake_result = result.copy()
fake_result["question"] = result.get("question", "What is the question?")
fake_result["ground_truth"] = "Christmas"
fake_result["contexts"] = ["I love christmas"]
fake_result["source_documents"] = [Document(page_content="I love christmas")]

# Evaluate using the context_recall_chain
eval_result = context_recall_chain.invoke(fake_result)

# Print the correct score
print("🎯 Context Recall Score:", eval_result.get("context_recall", "❌ Score not found"))


🎯 Context Recall Score: 0.0


2. `evaluate()`

Evaluate a list of inputs/queries and the outputs/predictions from the QA chain.

In [81]:
# Run the queries as a batch for efficiency
predictions = qa_chain.batch(examples)

# Evaluate
print("evaluating...")
r = []

for ex, pred in zip(examples, predictions):
    # Prepare input for EvaluatorChain
    input_data = {
        "question": ex["query"],
        "answer": pred["result"],
        "contexts": [doc.page_content for doc in pred["source_documents"]],
    }
    # Invoke evaluation
    result = faithfulness_chain.invoke(input_data)
    r.append(result)

r  # this will contain a list of evaluation results


evaluating...


[{'question': 'What is the population of New York City as of 2020?',
  'answer': 'The population of New York City as of 2020 is 8,804,190 residents.',
  'contexts': ["New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houston, and Phoenix, Arizona combined. New York City's population is about 44% of New York State's population, and about 39% of the population of the New York metropolitan area. The majority of New Y

In [85]:
# evaluate context recall
print("evaluating...")
r = []

for ex, pred in zip(examples, predictions):
    input_data = {
        "question": ex.get("query", "What is the question?"),
        "ground_truth": ex.get("answer", ex.get("result", "No ground truth")),
        "contexts": [doc.page_content for doc in pred.get("source_documents", [])],
    }
    result = context_recall_chain.invoke(input_data)
    r.append(result)

r


evaluating...


[{'question': 'What is the population of New York City as of 2020?',
  'ground_truth': 'No ground truth',
  'contexts': ["New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houston, and Phoenix, Arizona combined. New York City's population is about 44% of New York State's population, and about 39% of the population of the New York metropolitan area. The majority of New York City residents in 2020 (5,141,538, or 58.